In [3]:
import pandas as pd
import geopandas as gpd
import os
import json
import colorcet as cc
colour_palette = cc.CET_R3

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, reset_output

output_notebook()

### making data a bit easier to see
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### should be outside the repo'
data_location = r'/Users/jj/code/strava_data/'

Loading BokehJS ...

In [24]:
### will return a list of colours of n length from the colour_palette
def get_colours(n):
    distance = int(len(colour_palette) / n)
    return [x for i, x in enumerate(colour_palette) if i % distance == 0]

def aggregate(df, cat_columns=[], met_columns=[], aggregations=[]):
    raw_data = df[cat_columns+met_columns]
    grouped = raw_data.groupby(cat_columns).agg({m:aggregations for m in met_columns}).reset_index()
    grouped.columns = grouped.columns.map(lambda x: '_'.join(a for a in x if len(a)>0))
    print(grouped)

aggregate(raw, cat_columns=['type'], met_columns=['moving_time (seconds)', 'kudos_count'], aggregations=['max', 'sum'])

               type  moving_time (seconds)_max  moving_time (seconds)_sum  \
0         AlpineSki                      10800                     213012   
1              Hike                      28003                     513702   
2         NordicSki                      12857                      12857   
3              Ride                      41637                    3696301   
4               Run                      30345                     465999   
5         Snowboard                       5115                       7753   
6      StairStepper                        982                        982   
7   StandUpPaddling                       3546                       3546   
8              Swim                        830                       2416   
9              Walk                      19307                     534468   
10          Workout                      30789                     926970   
11             Yoga                       4572                     188236   

In [8]:
raw = pd.read_csv(data_location+'all_activities_cleaned.csv', index_col=0)
raw

,name,distance,moving_time (seconds),elapsed_time (seconds),total_elevation_gain,type,sport_type,workout_type,id,start_date_local,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,gear_id,start_latlng,end_latlng,average_speed (kph),max_speed (kph),average_temp,average_watts,kilojoules,device_watts,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,pr_count,total_photo_count,has_kudoed,suffer_score,average_cadence,max_watts,weighted_average_watts,athlete_id,athlete_resource_state,map_summary_polyline,bike_id,bike_primary,bike_name,bike_nickname,bike_retired,moving_time (minutes),moving_time (hours),elapsed_time (minutes),elapsed_time (hours),calories
0,Lunch Ride,2037.2,278,775,0.0,Ride,Ride,NaN,8726797885,2023-03-17 11:49:16,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.28829252906144, 174.7669970523566]","[-41.2935836892575, 174.77822947315872]",26.3808,56.9376,27.0,26.4,7.3,False,True,113.5,136.0,False,True,111.6,-14.0,0,0,False,0.0,NaN,NaN,NaN,23883723,1,zj_{Fggui`@DK@GS]KIyCiAeAe@}@QkA]kCeAc@UKOBg@D...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,4.633333,0.077222,12.916667,0.215278,2.0
1,Lunch Ride,2167.2,727,773,132.0,Ride,Ride,NaN,8721532095,2023-03-16 12:29:03,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.293124444782734, 174.77520930580795]","[-41.28730338066816, 174.76581109687686]",10.7316,19.8360,28.0,146.0,106.1,False,True,125.4,145.0,False,True,134.4,32.6,0,0,False,3.0,NaN,NaN,NaN,23883723,1,`a`{F_xvi`@?NQv@?d@I^Wn@]lAi@xBGb@i@xAKl@M`A[x...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,12.116667,0.201944,12.883333,0.214722,25.0
2,Lunch Ride,2109.1,267,302,3.0,Ride,Ride,NaN,8721256990,2023-03-16 11:56:29,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.28733104094863, 174.76634301245213]","[-41.29523408599198, 174.77632116526365]",28.4364,52.9488,28.0,33.5,8.9,False,True,105.8,121.0,False,True,114.0,2.0,0,0,False,0.0,NaN,NaN,NaN,23883723,1,nj_{Fggui`@BGIUOQkB_AqAs@_HeCq@]ES?QFg@JIf@ErA...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,4.450000,0.074167,5.033333,0.083889,2.0
3,Afternoon Mountain Bike Ride,33168.0,11823,20633,1089.9,Ride,MountainBikeRide,10.0,8716585507,2023-03-15 13:14:37,19,5,0,1,0,False,False,False,False,everyone,False,b11952969,"[-41.292270831763744, 174.77877404540777]","[-41.287265829741955, 174.76577061228454]",10.0980,44.8488,28.0,140.8,1664.1,False,True,144.5,183.0,False,True,183.1,7.7,8,0,False,159.0,NaN,NaN,NaN,23883723,1,v{_{Finwi`@fAwBlDgLbB_OjEwRsGsD{HiPpAiAvDOP~Aj...,b11952969,False,Canyon Spectral,Canyon Spectral,False,197.050000,3.284167,343.883333,5.731389,398.0
4,Morning Mountain Bike Ride,2300.3,449,525,2.0,Ride,MountainBikeRide,NaN,8715133554,2023-03-15 09:49:41,0,0,0,1,0,False,False,False,True,only_me,False,b4083088,"[-41.287781400606036, 174.76704055443406]","[-41.291415710002184, 174.77939656935632]",18.4428,48.5676,24.0,40.2,18.1,False,True,106.6,144.0,False,True,267.8,150.2,0,0,False,1.0,NaN,NaN,NaN,23883723,1,rj_{Fcgui`@BG?EGM_@W_@Og@YeDmAyEsBuAi@i@Bs@EIK...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,7.483333,0.124722,8.750000,0.145833,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2201,Radio dome ridge,19853.6,4766,4928,707.1,Ride,Ride,10.0,1121238293,2017-08-07 15:21:49,9,0,0,1,0,False,False,False,False,everyone,False,b4083088,"[-41.294885, 174.781158]","[-41.294907, 174.781201]",14.9976,71.2800,NaN,161.6,770.2,False,False,NaN,NaN,False,False,483.2,12.0,1,1,False,NaN,NaN,NaN,NaN,23883723,1,j{`{F_pwi`@|C~A`ADh@n@UlDeBdIdAp@Hf@cEnQUzB}@d...,b4083088,True,Avanti Giro 2,Avanti Giro 2,False,79.433333,1.323889,82.133333,1.368889,184.0
2202,Morning Ride,37396.5,5493,5586,670.4,Ride,Ride,10.0,1117419963,2017-

In [192]:
if os.path.isfile(data_location+'athlete_info_jj.json'):
    athlete_info = json.load(open(data_location+'athlete_info_jj.json'))
    bikes = {x['id']: x['name'] for x in athlete_info['bikes']}
    raw['gear_id'] = raw['gear_id'].map(bikes)
    

In [68]:
def bar_chart_count(s):
    agged = s.value_counts()
    x = agged.index.to_list()
    y = agged.to_list()
    c = get_colours(len(x)-1)
    title = s.name[0].upper()+s.name[1:]+'s of Activities'

    p = figure(x_range = x, title=title, background_fill_color='whitesmoke', height=300)
    p.vbar(x=x, top=y, width=.9, fill_color=c, line_color='white', line_width=3, line_join='round')

    p.y_range.start = 0
    p.y_range.end = max(y) * 1.05
    p.xgrid.grid_line_color=None
    p.xaxis.major_label_orientation = .8
    p.yaxis.minor_tick_line_color = None

    show(p)

bar_chart_count(raw['type'])


In [180]:
bar_chart_count(raw['gear_id'])

In [181]:
def bar_chart_sum(cat_col, num_col, sort_by_values=True):

    agged = num_col.groupby(cat_col).sum().sort_values(ascending=False)
    if sort_by_values == False:
        agged = agged.sort_index()
    x = agged.index.astype('string').to_list()
    y = agged.to_list()
    c = get_colours(len(x)) [:len(x)]
    title = num_col.name[0].upper()+num_col.name[1:]+ ' by ' + cat_col.name[0].upper()+cat_col.name[1:]

    p = figure(x_range = x, title=title, background_fill_color='whitesmoke', height=300)
    p.vbar(x=x, top=y, width=.9, fill_color=c, line_color='white', line_width=3, line_join='round')

    p.y_range.start = 0
    p.y_range.end = max(y) * 1.05
    p.xgrid.grid_line_color=None
    p.xaxis.major_label_orientation = .8
    p.yaxis.minor_tick_line_color = None

    show(p)

bar_chart_sum(raw['gear_id'], raw['distance'] / 1000)

In [111]:
bar_chart_sum(raw['start_date_local'].astype('datetime64[ns]').dt.year, raw['distance'] / 1000, sort_by_values=False)

In [182]:
def stacked_bar_chart(bar_col, legend_col):

    agged = bar_col.groupby([legend_col, bar_col]).count().unstack().fillna(0)
    bars = agged.index.to_list()
    legends = agged.columns.to_list()
    c = get_colours(len(legends))


    data = {l:agged[l].to_list() for l in legends}

    c = get_colours(len(agged.columns))
    title = 'Activities by '+bar_col.name[0].upper() + bar_col.name[1:] + ' and ' + legend_col.name[0].upper() + legend_col.name[1:]

    p = figure(x_range=bars, title=title, background_fill_color='whitesmoke', height=300)
    bottom = [0 for x in bars]
    for i, l in enumerate(legends):
        top = [x+y for x, y in zip(bottom, data[l])]
        p.vbar(x=bars, bottom=bottom, top=top, width=.9, color=c[i], line_color='white', line_width=1, line_join='round', legend_label=l)
        bottom = top

    p.y_range.start = 0
    p.y_range.end = max(agged.sum(axis=1)) * 1.05
    p.xgrid.grid_line_color=None
    p.xaxis.major_label_orientation = .8
    p.yaxis.minor_tick_line_color = None
    p.legend.location = 'top_left'

    show(p)

stacked_bar_chart(raw['gear_id'], raw['start_date_local'].astype('datetime64[ns]').dt.year.astype('string'))

In [187]:
def stacked_bar_chart_sum(bar_col, legend_col, num_col):

    agged = num_col.groupby([legend_col, bar_col]).sum().unstack().fillna(0)
    bars = agged.index.to_list()
    legends = agged.columns.to_list()
    c = get_colours(len(legends))


    data = {l:agged[l].to_list() for l in legends}

    c = get_colours(len(agged.columns))
    title = num_col.name[0].upper() + num_col.name[1:] + ' by '+bar_col.name[0].upper() + bar_col.name[1:] + ' and ' + legend_col.name[0].upper() + legend_col.name[1:]

    p = figure(x_range=bars, title=title, background_fill_color='whitesmoke', height=400)
    bottom = [0 for x in bars]
    for i, l in enumerate(legends):
        top = [x+y for x, y in zip(bottom, data[l])]
        p.vbar(x=bars, bottom=bottom, top=top, width=.9, color=c[i], line_color='white', line_width=1, line_join='round', legend_label=l)
        bottom = top

    p.y_range.start = 0
    p.y_range.end = max(agged.sum(axis=1)) * 1.2
    p.xgrid.grid_line_color=None
    p.xaxis.major_label_orientation = .8
    p.yaxis.minor_tick_line_color = None
    p.legend.location = 'top_left'
    p.legend.label_text_font_size = '8pt'

    show(p)

stacked_bar_chart_sum(raw['gear_id'], raw['start_date_local'].astype('datetime64[ns]').dt.year.astype('string'), raw['distance'] / 1000)

In [188]:
stacked_bar_chart_sum(raw['gear_id'], raw['start_date_local'].astype('datetime64[ns]').dt.year.astype('string'), raw['moving_time'] / 3600)

In [193]:
stacked_bar_chart_sum(raw['gear_id'], raw['start_date_local'].astype('datetime64[ns]').dt.year.astype('string'), raw['total_elevation_gain'])

In [194]:
stacked_bar_chart_sum(raw['gear_id'], raw['start_date_local'].astype('datetime64[ns]').dt.year.astype('string'), raw['kudos_count'])

In [200]:
stacked_bar_chart_sum(raw['type'], raw['start_date_local'].astype('datetime64[ns]').dt.year.astype('string'), raw['moving_time'] / 60 / 60)